In [24]:
import json
import nlpaug.augmenter.sentence as nas
import nlpaug.augmenter.word as naw
import nlpaug.flow.sequential as naf
import torch

### Data Augmentation - En-De Round-trip Translation - Top10

In [ ]:
!git clone https://github.com/pytorch/fairseq.git fairseq-git
!cd fairseq-git
!pip3 install sacrebleu
!pip3 install --editable .
# !python setup.py build develop

In [ ]:
!pip3 install bitarray fastBPE hydra-core omegaconf regex requests sacremoses subword_nmt

In [ ]:
import os
import json
import torch

In [ ]:
current_path = os.getcwd()
os.chdir('/content/fairseq')
os.getcwd()

In [69]:
with open('./new_train_test_labeled_dataset.json','r') as f:
    data = json.load(f)

In [ ]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')
en2de.cuda()
de2en.cuda()  

In [76]:
def round_trip_translation(sent):
    ende_rt_translation = []
    en_toks = en2de.tokenize(sent)
    en_bpe = en2de.apply_bpe(en_toks)
    en_bin = en2de.binarize(en_bpe)
    de_bin = en2de.generate(en_bin, beam=10, sampling=True, sampling_topk=10)
    for b in de_bin:
        de_sample = b['tokens']
        de_bpe = en2de.string(de_sample)
        de_toks = en2de.remove_bpe(de_bpe)
        de = en2de.detokenize(de_toks)
        trans = de2en.translate(de)
        ende_rt_translation.append(trans)
    return ende_rt_translation

In [71]:
len(data['document_train'])

356

In [79]:
%%time

augmented_doc_train = []
augmented_summ_train = []

for i, (d, s) in enumerate(zip(data['document_train'], data['summary_train'])):
    augmented_doc_train += round_trip_translation(d)
    augmented_summ_train += round_trip_translation(s)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

### Back Translation - TopK

##### Most suitable for process data as it doesn't change the structure
##### Document Augmentation

In [2]:
with open('./final_new_labeled_dataset.json','r') as f:
    data = json.load(f)

In [30]:
# Round-trip translations between English and German:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe)
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe)


Using cache found in /Users/I543118/.cache/torch/hub/pytorch_fairseq_main
Loading codes from /Users/I543118/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3/bpecodes ...
Read 30000 codes from the codes file.
Using cache found in /Users/I543118/.cache/torch/hub/pytorch_fairseq_main
100%|██████████████████████| 2992273886/2992273886 [04:05<00:00, 12188490.17B/s]
Loading codes from /Users/I543118/.cache/torch/pytorch_fairseq/f42bb1b72d293668a5c50d9589fd2f3cc27322e390b1ef4cf3fdcf625c0d2fd7.bf6e22453272c2cba218a5ccecd045f73e926c34c1d66c47c9b31233343820a9/bpecodes ...
Read 30000 codes from the codes file.


In [31]:
# Compare the results with English-Russian round-trip translation:
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')


Using cache found in /Users/I543118/.cache/torch/hub/pytorch_fairseq_main
100%|██████████████████████| 3037373614/3037373614 [03:35<00:00, 14106854.55B/s]
Loading codes from /Users/I543118/.cache/torch/pytorch_fairseq/57b24ff1a88d5f14807352c9d8e17949d46ae3d0a06c1be3667929101011cdc0.8699fa0753ba14331352cc26ceb7aa46fba856bad52dd40fc97d3834418bf6f1/bpecodes ...
Read 30000 codes from the codes file.
Using cache found in /Users/I543118/.cache/torch/hub/pytorch_fairseq_main
100%|██████████████████████| 3041223502/3041223502 [04:11<00:00, 12115315.41B/s]
Loading codes from /Users/I543118/.cache/torch/pytorch_fairseq/f857729c1dcbbade3bbe70f7f58ede13e939a7b8278eb9015602e73be09e5cb4.ecb440f9c8cec1f28ee5023cdeab76f67eede4fc1ac0f2c70eeb2e65a25ad783/bpecodes ...
Read 24000 codes from the codes file.


In [46]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /Users/I543118/.cache/torch/hub/pytorch_fairseq_main
Loading codes from /Users/I543118/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3/bpecodes ...
Read 30000 codes from the codes file.


In [47]:
# Manually tokenize:
en_toks = en2de.tokenize(data['document'][0])
# Manually apply BPE:
en_bpe = en2de.apply_bpe(en_toks)
# Manually binarize:
en_bin = en2de.binarize(en_bpe)

# Generate five translations with top-k sampling:
de_bin = en2de.generate(en_bin, beam=10, sampling=True, sampling_topk=10)

In [57]:
len(de_bin)

10

In [52]:
# Convert one of the samples to a string and detokenize
de_sample = de_bin[0]['tokens']
de_bpe = en2de.string(de_sample)
de_toks = en2de.remove_bpe(de_bpe)
de = en2de.detokenize(de_toks)
assert de == en2de.decode(de_sample)

In [55]:
de2en.translate(de)

'Application reception, extract and conversion gpa, check plagiarism of cover letter, read and evaluate cover letter, provisional ranking assigned, application marked as "failed"'

In [59]:
%%time

ende_rt_translation = []
d = data['document'][0]
en_toks = en2de.tokenize(d)
en_bpe = en2de.apply_bpe(en_toks)
en_bin = en2de.binarize(en_bpe)
de_bin = en2de.generate(en_bin, beam=10, sampling=True, sampling_topk=10)
for b in de_bin:
    de_sample = b['tokens']
    de_bpe = en2de.string(de_sample)
    de_toks = en2de.remove_bpe(de_bpe)
    de = en2de.detokenize(de_toks)
    trans = de2en.translate(de)
    ende_rt_translation.append(trans)

CPU times: user 49.9 s, sys: 768 ms, total: 50.7 s
Wall time: 50.3 s


In [65]:
print('Ex 1: '+ data['document'][0] + '\n\nAugmented:')
ende_rt_translation

Ex 1: application received, extract and convert gpa, check plagiarism of motivation letter, read and score motivation letter, preliminary ranking assigned, application marked as "failed"

Augmented:


['Receive registration, extract and convert GPA, check plagiarism of cover letter, read and evaluate cover letter, assign preliminary ranking, mark application as "failed"',
 'Application documents, extract and conversion of the gpa, plagiarism examination of the cover letter, reading and evaluating cover letters, provisional ranking assigned, cover letter marked "failed"',
 'Submit application documents, extract and convert, check plagiarism of a cover letter, read and evaluate cover letter, assign provisional ranking, mark application as "failed"',
 'Submit, extract and convert application documents, check cover letters for plagiarism protection, read and evaluate cover letters, assign provisional ranking, mark application as "failure"',
 'Receive application documents, extract and implement GPA, check plagiarism from cover letters, read and evaluate cover letters, assign first ranking, mark application with "failed"',
 'Receive application, if applicable extract and conversion, chec

In [66]:
%%time

enru_rt_translation = []
d = data['document'][0]
en_toks = en2ru.tokenize(d)
en_bpe = en2ru.apply_bpe(en_toks)
en_bin = en2ru.binarize(en_bpe)
ru_bin = en2ru.generate(en_bin, beam=10, sampling=True, sampling_topk=10)
for b in ru_bin:
    ru_sample = b['tokens']
    ru_bpe = en2ru.string(ru_sample)
    ru_toks = en2ru.remove_bpe(ru_bpe)
    ru = en2ru.detokenize(ru_toks)
    trans = ru2en.translate(ru)
    enru_rt_translation.append(trans)

In [68]:
print('Ex 1: '+ data['document'][0] + '\n\nAugmented:')
enru_rt_translation

Ex 1: application received, extract and convert gpa, check plagiarism of motivation letter, read and score motivation letter, preliminary ranking assigned, application marked as "failed"

Augmented:


['application received, gpa extrapolation and conversion, verification for plagiarism of motivational letter, reading and evaluation of motivational letter, assignment of preliminary rating, application marked "failed"',
 'application received, extrapolated and converted to gpa, plagiarism of motivational letter checked, incentive letter read and evaluated, provisional rating assigned, application marked as "failed"',
 'application to receive, extrapolate and convert gpa, check for plagiarism of motivational letter, view and evaluate motivational letter, assign preliminary rating, mark application as "rejected"',
 'received the application, extrapolation and conversion of the gp, verification of the plagiarism of the motivational letter, reading and evaluation of the motivational letter, assigning a preliminary rating, the application is marked as "failed"',
 'application received, extrapolated and converted to gpa, check the plagiarism of the motivational letter, read and evaluate the

In [3]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',
    to_model_name='facebook/wmt19-de-en')

In [4]:
augmented_data = back_translation_aug.augment(data['document'][0])
print('Ex 1: '+ data['document'][0] + '\n\nAugmented: ' + augmented_data + '\n')
# will need to also lower case the output

Ex 1: application received, extract and convert gpa, check plagiarism of motivation letter, read and score motivation letter, preliminary ranking assigned, application marked as "failed"

Augmented: Application received, extract and convert gpa, check for plagiarism of the motivation letter, read and evaluate the motivation letter, assign preliminary ranking, application marked as "failed"



In [5]:
augmented_data = back_translation_aug.augment(data['document'][4])
print('Ex 2: '+ data['document'][4] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 2: payment processed, draw up completion mail, wait for arrival of customer in studio, after 7 days , pick-up reminder sent , hand over printouts, pack and send printouts, dropbox link added to completion mail, voucher added to completion mail, completion mail drawn up

Augmented: Payment processing, E-mail completion, Waiting for the customer to arrive in the studio, after 7 days, Collection of the reminder, Delivery of the printouts, Packaging and sending of the printouts, Dropbox link to the completion e-mail, Voucher for completion e-mail, Completion e-mail created



In [6]:
augmented_data = back_translation_aug.augment(data['document'][3])
print('Ex 3: '+ data['document'][3] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 3: decide application offer, declination message, acceptation message, wait for kul if higher ranked applicants decline offer, application ended

Augmented: Application offer, Rejection message, Acceptance message, Waiting for Kul, if higher ranking applicants reject the offer, Application terminated



##### Summary Augmentation

In [8]:
augmented_data = back_translation_aug.augment(data['summary'][0])
print('Ex 1: '+ data['summary'][0] + '\n\nAugmented : ' + augmented_data + '\n')

Ex 1: assign preliminary ranking for incoming application

Augmented : Determine preliminary order of priority for incoming applications



In [9]:
augmented_data = back_translation_aug.augment(data['summary'][4])
print('Ex 2: '+ data['summary'][4] + '\n\nAugmented : ' + augmented_data + '\n')
# for None outcome -> use original or try other translation model

Ex 1: finish order

Augmented : 



In [10]:
augmented_data = back_translation_aug.augment(data['summary'][3])
print('Ex 3: '+ data['summary'][3] + '\n\nAugmented : ' + augmented_data + '\n')

Ex 1: decide application offer

Augmented : Decide on the offer



### Sentence Level Augmentation

#### Abstractive Summary Augmentation
##### Document Augmentation - can set threshold, num of document sentences >= i.e. 5, can apply abstractive summary

In [4]:
abst_summ_aug = nas.AbstSummAug(model_path='t5-base')

In [8]:
augmented_data = abst_summ_aug.augment(data['document'][0])
print('Ex 1: '+ data['document'][0] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: application received, extract and convert gpa, check plagiarism of motivation letter, read and score motivation letter, preliminary ranking assigned, application marked as "failed"

Augmented: application received, extract and convert gpa, check plagiarism of motivation letter.



In [9]:
augmented_data = abst_summ_aug.augment(data['document'][4])
print('Ex 2: '+ data['document'][4] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: payment processed, draw up completion mail, wait for arrival of customer in studio, after 7 days , pick-up reminder sent , hand over printouts, pack and send printouts, dropbox link added to completion mail, voucher added to completion mail, completion mail drawn up

Augmented: payment processed, draw up completion mail, wait for arrival of customer in studio. after 7 days. pick-up reminder sent, hand over printouts, pack and send printout. dropbox link added to completion mail and voucher added.



In [10]:
augmented_data = abst_summ_aug.augment(data['document'][3])
print('Ex 3: '+ data['document'][3] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: decide application offer, declination message, acceptation message, wait for kul if higher ranked applicants decline offer, application ended

Augmented: kul: higher ranked applicants decline offer, application ended. decision made, declination message, acceptance message, wait for kul if kul declines offer.



##### Summary Augmentation - Not suitable for label part as the structure changes - Verb + Object

In [5]:
abst_summ_aug_len10 = nas.AbstSummAug(model_path='t5-base', max_length=10)

In [6]:
augmented_data = abst_summ_aug_len10.augment(data['summary'][0])
print('Ex 1: '+ data['summary'][0] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: assign preliminary ranking for incoming application

Augmented: Applicants are ranked according to preliminary ranking. preliminary ranking is based on the number of applications received.



In [7]:
augmented_data = abst_summ_aug_len10.augment(data['summary'][4])
print('Ex 2: '+ data['summary'][4] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: finish order

Augmented: finish order



In [8]:
augmented_data = abst_summ_aug_len10.augment(data['summary'][3])
print('Ex 3: '+ data['summary'][3] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: decide application offer

Augmented: : decide application offer offer:.



#### Sentence Insertion 
##### Not suitable as it changes the sentence structure - Train set distribution might change and deviate too much from the test set distribution
##### Document Augmentation

In [9]:
sent_inset_aug = nas.ContextualWordEmbsForSentenceAug(model_path='xlnet-base-cased')

In [10]:
augmented_data = sent_inset_aug.augment(data['document'][0])
print('Ex 1: '+ data['document'][0] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: application received, extract and convert gpa, check plagiarism of motivation letter, read and score motivation letter, preliminary ranking assigned, application marked as "failed"

Augmented: application received, extract and convert gpa, check plagiarism of motivation letter, read and score motivation letter, preliminary ranking assigned, application marked as "failed" or no certification or credential for successful performance of the application.



In [11]:
augmented_data = sent_inset_aug.augment(data['document'][4])
print('Ex 2: '+ data['document'][4] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: payment processed, draw up completion mail, wait for arrival of customer in studio, after 7 days , pick-up reminder sent , hand over printouts, pack and send printouts, dropbox link added to completion mail, voucher added to completion mail, completion mail drawn up

Augmented: payment processed, draw up completion mail, wait for arrival of customer in studio, after 7 days , pick-up reminder sent , hand over printouts, pack and send printouts, dropbox link added to completion mail, voucher added to completion mail, completion mail drawn up .



In [12]:
augmented_data = sent_inset_aug.augment(data['document'][3])
print('Ex 3: '+ data['document'][3] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: decide application offer, declination message, acceptation message, wait for kul if higher ranked applicants decline offer, application ended

Augmented: decide application offer, declination message, acceptation message, wait for kul if higher ranked applicants decline offer, application ended after 45 hours after 60 applications were filled by 35% on 7 days after 20 days at this company and in other business at a wide ranging level for



##### Summary Augmentation

In [13]:
augmented_data = sent_inset_aug.augment(data['summary'][0])
print('Ex 1: '+ data['summary'][0] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 1: assign preliminary ranking for incoming application

Augmented: assign preliminary ranking for incoming application category after 48 Days.



In [14]:
augmented_data = sent_inset_aug.augment(data['summary'][4])
print('Ex 2: '+ data['summary'][4] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 2: finish order

Augmented: finish order is only very important all through a good cooking experience if well crafted to create the perfect version or even your dinner if perfectly finished once.



In [15]:
augmented_data = sent_inset_aug.augment(data['summary'][3])
print('Ex 3: '+ data['summary'][3] + '\n\nAugmented: ' + augmented_data + '\n')

Ex 3: decide application offer

Augmented: decide application offer is too generous is our choice?



### Word Level Augmentation
#### Synonym Replacement & Random Deletion 

In [20]:
aug_syn = naw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=1, aug_max=10, aug_p=0.3, lang='eng', 
                     stopwords=None, tokenizer=None, reverse_tokenizer=None, stopwords_regex=None, force_reload=False, 
                     verbose=0)

In [18]:
TOPK=20 #default=100
ACT = 'insert' #"substitute"
aug_bert = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', 
    #device='cuda',
    action=ACT, top_k=TOPK)

In [13]:
aug_w2v = naw.WordEmbsAug(
    model_type='glove', model_path='./glove/glove.6B.300d.txt',
    action="substitute")

### Flow Augmentation

In [15]:
data['document'][3]

'decide application offer, declination message, acceptation message, wait for kul if higher ranked applicants decline offer, application ended'

In [22]:
aug = naf.Sequential([
    aug_bert,aug_w2v
])

aug.augment(data['document'][3], n=10)

['appoint application only, seek declination message, plea adaina message, wait subsequent for kul validation fact higher eight ranked visa applicants decline the offer, application service opening',
 'decide application options, declination acceptance message, acceptation message, just application for kul examination again rank higher others sixth applicants supported decline any offer, entry yet 1997',
 'if decide integrated ask, necessarily declination probably message, internationality message, wait wait application for solutions kul if two higher ranked applicants decline admittance offer, apply ended',
 'try decide technology seeking, declination take wish, acceptation wait message, wait deadline two licenses kul if below ranked applicants simply weaker to buy, application offers ended',
 'decision application server offer, choose declination status text, sekirei message, select wait application for kayn if sales ranked ineligible must reversing offer, decides application congreg

In [ ]:
# flow_aug = naf.Sequential([
#     back_translation_aug, abst_summ_aug
# ])

# flow_aug.augment(data['document'][3], n=10)

### Augmentation Strategy
##### whether validation set is needed to decide 1:n and augmentation combinations